In [1]:
import findspark

In [2]:
findspark.init('C:\Spark')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("Reg_model").getOrCreate()

In [5]:
df = spark.read.csv("reg_data.csv", inferSchema = True, header = True)

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [7]:
df

DataFrame[Ship_name: string, Cruise_line: string, Age: int, Tonnage: double, passengers: double, length: double, cabins: double, passenger_density: double, crew: double]

In [8]:
df.groupBy("Cruise_line").count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [9]:
from pyspark.sql import SQLContext
#from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import Vectors, VectorUDT

In [10]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line",outputCol="Cruise_category")
indexed = indexer.fit(df).transform(df)

In [11]:
df.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0)]

In [12]:
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_category|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [13]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [14]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_category']

In [15]:
assembler = VectorAssembler(
    inputCols = ['Age',
                 'Tonnage',
                 'passengers',
                 'length',
                 'cabins',
                 'passenger_density',
                 'Cruise_category'],
    outputCol="features")

In [16]:
output = assembler.transform(indexed)

In [17]:
data = output.select("features", "crew")


In [18]:
data.select("features","crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [19]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [20]:
test_data.head(10)

[Row(features=DenseVector([4.0, 220.0, 54.0, 11.82, 27.0, 40.74, 0.0]), crew=21.0),
 Row(features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]), crew=3.55),
 Row(features=DenseVector([6.0, 90.0, 20.0, 9.64, 10.29, 45.0, 15.0]), crew=9.0),
 Row(features=DenseVector([7.0, 116.0, 31.0, 9.51, 15.57, 37.42, 2.0]), crew=12.0),
 Row(features=DenseVector([8.0, 91.0, 22.44, 9.65, 11.22, 40.55, 4.0]), crew=11.0),
 Row(features=DenseVector([8.0, 110.0, 29.74, 9.51, 14.87, 36.99, 1.0]), crew=11.6),
 Row(features=DenseVector([9.0, 59.058, 17.0, 7.63, 8.5, 34.74, 7.0]), crew=7.4),
 Row(features=DenseVector([9.0, 90.09, 25.01, 9.62, 10.94, 36.02, 0.0]), crew=8.69),
 Row(features=DenseVector([9.0, 110.0, 29.74, 9.52, 14.87, 36.99, 1.0]), crew=11.6),
 Row(features=DenseVector([9.0, 113.0, 26.74, 9.51, 13.37, 42.26, 2.0]), crew=12.38)]

In [21]:
from pyspark.ml.regression import LinearRegression

In [22]:
lr = LinearRegression(labelCol="crew")

In [23]:
model= lr.fit(train_data)

In [24]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [25]:
model.coefficients

DenseVector([-0.0182, 0.0071, -0.1245, 0.4096, 0.8294, 0.0056, 0.0508])

In [26]:
predicted_results = model.evaluate(test_data)

In [27]:
predicted_results.rootMeanSquaredError

0.8815343229982642

In [28]:
from pyspark.sql.functions import corr

In [29]:
df.select(corr("crew","passengers")).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [30]:
predictionsA = model.transform(test_data)
display(predictionsA)

DataFrame[features: vector, crew: double, prediction: double]

In [31]:
model

LinearRegression_44ec889b9b19a49cb5fa